## Setup

In [33]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
# load model architecture
from tep.modelUtils import load_architecture
model = load_architecture('models/tl/baseline.json')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
seq_input (InputLayer)       (None, 32)                0         
_________________________________________________________________
word_emb (Embedding)         (None, 32, 100)           43224100  
_________________________________________________________________
input_dropout (Dropout)      (None, 32, 100)           0         
_________________________________________________________________
padding_1 (ZeroPadding1D)    (None, 34, 100)           0         
_________________________________________________________________
conv_1 (Conv1D)              (None, 32, 128)           38528     
_________________________________________________________________
pool_1 (MaxPooling1D)        (None, 16, 128)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 128)           0         
__________

In [2]:
# set model weights
model.load_weights('models/tl/baseline.hdf5')

In [4]:
# load data and labels
from tep.utils import load_array
seqs = load_array('data/sentiment_data/seq_32.bc')
labels = load_array('data/sentiment_data/labels.bc')
print(seqs.shape)
print(labels.shape)

(1600000, 32)
(1600000,)


In [8]:
# load word index
import json
word_index_file = open('data/sentiment_data/word_index.json')
word_index_str = word_index_file.read()
word_index = json.loads(word_index_str)
type(word_index)

dict

In [9]:
# compile the model
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['acc'])

In [11]:
# test the model
from tep.trainUtils import one_hot_encoding
oh_labels = one_hot_encoding(labels, 2)
model.evaluate(x=seqs[:10000], y=oh_labels[:10000], batch_size=256)

10000/10000 [==============================] - 2s 212us/step


[0.34918261456489563, 0.84550000000000003]

## Get hidden layer activations

In [43]:
from tep.featureVisualization import FeatureVisualizer
fv = FeatureVisualizer(model, word_index)

In [44]:
activations = fv.get_activations('conv_1', seqs[:1])

In [45]:
type(activations)

numpy.ndarray

In [46]:
activations.shape

(1, 32, 128)

In [47]:
seq = seqs[0]

In [48]:
seq

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      2, 150999,
           12,    148,   4962, 101456,     10,    286,     20,    363], dtype=int32)

In [53]:
fv.get_words(seq, 30, 3)

['birthday', 'on', 'monday']